In [50]:
import pandas as pd
from rdkit import Chem

In [51]:
data = pd.read_csv("data.csv");

/Users/daraakdag/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
descriptors = pd.read_csv("descriptors.csv");

In [53]:
data.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Phenotype,Potency,Efficacy,...,brca1-p1_activity-Activity at 4.610 uM,brca1-p1_activity-Activity at 6.450 uM,brca1-p1_activity-Activity at 9.218 uM,brca1-p1_activity-Activity at 12.69 uM,brca1-p1_activity-Activity at 18.70 uM,brca1-p1_activity-Activity at 36.87 uM,brca1-p1_activity-Activity at 46.10 uM,brca1-p1_activity-Activity at 64.71 uM,brca1-p1_activity-Activity at 92.20 uM,Compound QC
0,RESULT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,STRING,FLOAT,FLOAT,...,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,STRING
1,RESULT_DESCR,NaN,NaN,NaN,NaN,NaN,NaN,"Indicates type of activity observed: inhibitor, activator, fluorescent, cytotoxic, inactive, or inconclusive.","Concentration at which compound exhibits half-maximal efficacy, AC50. Extrapolated AC50s also include the highest efficacy observed and the concentration of compound at which it was observed.","Maximal efficacy of compound, reported as a percentage of control. These values are estimated based on fits of the Hill equation to the dose-response curves.",...,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,% Activity at given concentration.,"NCGC designation for data stage: 'qHTS', 'qHTS Verification', 'Secondary Profiling'"
2,RESULT_UNIT,NaN,NaN,NaN,NaN,NaN,NaN,NONE,MICROMOLAR,PERCENT,...,PERCENT,PERCENT,PERCENT,PERCENT,PERCENT,PERCENT,PERCENT,PERCENT,PERCENT,NONE
3,RESULT_IS_ACTIVE_CONCENTRATION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRUE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RESULT_ATTR_CONC_MICROMOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.61,6.45,9.21842,12.687,18.7029,36.8711,46.0969,64.715,92.2,NaN


In [54]:
data = data.loc[5:,['PUBCHEM_SID','PUBCHEM_ACTIVITY_SCORE']];

In [71]:
inactives= data.loc[data.PUBCHEM_ACTIVITY_SCORE==0]
actives= data.loc[data.PUBCHEM_ACTIVITY_SCORE!=0]
print("Number of inactives is ",len(inactives))
print("Number of actives is ", len(actives))

Number of inactives is  364035
Number of actives is  13515


In [80]:
selectedInactives = inactives.sample(350)
selectedActives = actives.sample(700)

In [81]:
smallDataset = pd.concat([selectedInactives,selectedActives])

In [88]:
print("Shape of small dataset is",smallDataset.shape)

Shape of small dataset is (1050, 2)


In [89]:
smallDataset.head()

,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE
231584,47197529.0,0.0
375920,124879998.0,0.0
358318,99360986.0,0.0
204552,26657698.0,0.0
119006,22403948.0,0.0
